In [15]:
from queue import PriorityQueue

In [16]:
initial_config = [[2, 8, 3], [1, 6, None], [4, 7, 5]]
goal_config = [[1, 2, 3], [8, None, 4], [7, 6, 5]]

In [17]:
class PuzzleState:
    def __init__(self, puzzle, parent=None, move=None, depth=0):
        self.puzzle = tuple(map(tuple, puzzle))  # Convertendo para tupla
        self.parent = parent
        self.move = move
        self.depth = depth

    def __eq__(self, other):
        return self.puzzle == other.puzzle

    def __hash__(self):
        return hash(self.puzzle)

    def __str__(self):
        return "\n".join([" ".join(map(str, row)) for row in self.puzzle])
    
    def __lt__(self, other):
        return self.depth < other.depth

def generate_successors(state):
    successors = []
    empty_row, empty_col = next((i, j) for i, row in enumerate(state.puzzle) for j, val in enumerate(row) if val is None)

    moves = [(0, 1), (0, -1), (1, 0), (-1, 0)]

    for dr, dc in moves:
        new_row, new_col = empty_row + dr, empty_col + dc

        if 0 <= new_row < 3 and 0 <= new_col < 3:
            new_puzzle = [list(row) for row in state.puzzle]
            new_puzzle[empty_row][empty_col], new_puzzle[new_row][new_col] = new_puzzle[new_row][new_col], None

            successors.append(PuzzleState(new_puzzle, parent=state, move=(empty_row, empty_col), depth=state.depth + 1))

    return successors

In [18]:
def manhattan_distance(puzzle, goal):
    distance = 0

    for i, row in enumerate(puzzle):
        for j, val in enumerate(row):
            if val is not None:
                goal_row, goal_col = next((x, y) for x, r in enumerate(goal) for y, v in enumerate(r) if v == val)
                distance += abs(i - goal_row) + abs(j - goal_col)

    return distance


def greedy_search(initial_state, goal_state):
    frontier = PriorityQueue()
    explored = set()

    frontier.put((manhattan_distance(initial_state.puzzle, goal_state.puzzle), initial_state))

    while not frontier.empty():
        _, current_state = frontier.get()

        if current_state == goal_state:
            path = []
            while current_state:
                path.append(current_state)
                current_state = current_state.parent
            path.reverse()
            return path

        explored.add(current_state)

        successors = generate_successors(current_state)
        for successor in successors:
            if successor not in explored:
                priority = manhattan_distance(successor.puzzle, goal_state.puzzle)
                frontier.put((priority, successor))

    return None 


In [20]:
manhattan_distance(initial_config, goal_config)

9

In [19]:
initial_state = PuzzleState(initial_config)
goal_state = PuzzleState(goal_config)

solution_path = greedy_search(initial_state, goal_state)

if solution_path:
    print("Solution found in", solution_path[-1].depth, "moves:")
    for step in solution_path:
        print(f'{step}\n')
else:
    print("No solution found.")


Solution found in 21 moves:
2 8 3
1 6 None
4 7 5

2 8 3
1 None 6
4 7 5

2 None 3
1 8 6
4 7 5

None 2 3
1 8 6
4 7 5

1 2 3
None 8 6
4 7 5

1 2 3
4 8 6
None 7 5

1 2 3
4 8 6
7 None 5

1 2 3
4 None 6
7 8 5

1 2 3
4 6 None
7 8 5

1 2 3
4 6 5
7 8 None

1 2 3
4 6 5
7 None 8

1 2 3
4 None 5
7 6 8

1 2 3
None 4 5
7 6 8

1 2 3
7 4 5
None 6 8

1 2 3
7 4 5
6 None 8

1 2 3
7 4 5
6 8 None

1 2 3
7 4 None
6 8 5

1 2 3
7 None 4
6 8 5

1 2 3
7 8 4
6 None 5

1 2 3
7 8 4
None 6 5

1 2 3
None 8 4
7 6 5

1 2 3
8 None 4
7 6 5

